# Milp


In [27]:
import docplex.mp.model as mp
from cplex import infinity
import numpy as np
import tensorflow as tf
import pandas as pd


def codify_network_fischetti(
    mdl,
    layers,
    input_variables,
    auxiliary_variables,
    intermediate_variables,
    decision_variables,
    output_variables,
):
    output_bounds = []
    bounds = []

    for i in range(len(layers)):
        A = layers[i].get_weights()[0].T
        b = layers[i].bias.numpy()
        x = input_variables if i == 0 else intermediate_variables[i - 1]
        if i != len(layers) - 1:
            s = auxiliary_variables[i]
            a = decision_variables[i]
            y = intermediate_variables[i]
        else:
            y = output_variables

        for j in range(A.shape[0]):
            if i != len(layers) - 1:
                mdl.add_constraint(
                    A[j, :] @ x + b[j] == y[j] - s[j], ctname=f"c_{i}_{j}"
                )
                mdl.add_indicator(a[j], y[j] <= 0, 1)
                mdl.add_indicator(a[j], s[j] <= 0, 0)

                mdl.maximize(y[j])
                mdl.solve()
                ub_y = mdl.solution.get_objective_value()
                mdl.remove_objective()

                mdl.maximize(s[j])
                mdl.solve()
                ub_s = mdl.solution.get_objective_value()
                mdl.remove_objective()

                y[j].set_ub(ub_y)
                s[j].set_ub(ub_s)

                bounds.append([-ub_s, ub_y])

            else:
                mdl.add_constraint(A[j, :] @ x + b[j] == y[j], ctname=f"c_{i}_{j}")
                mdl.maximize(y[j])
                mdl.solve()
                ub = mdl.solution.get_objective_value()
                mdl.remove_objective()

                mdl.minimize(y[j])
                mdl.solve()
                lb = mdl.solution.get_objective_value()
                mdl.remove_objective()

                y[j].set_ub(ub)
                y[j].set_lb(lb)
                output_bounds.append([lb, ub])
                
                bounds.append([lb, ub])

    return mdl, output_bounds, bounds


# todo: ver se faz uma chamada para cada classe não predita
def codify_network_fischetti_relaxed(
    mdl,
    layers,
    input_variables,
    auxiliary_variables,
    intermediate_variables,
    decision_variables,
    output_variables,
    output_bounds_binary_variables,
    bounds = []
):
    output_bounds = []

    for i in range(len(layers)):  # para cada camada
        A = layers[i].get_weights()[0].T
        b = layers[i].bias.numpy()
        x = input_variables if i == 0 else intermediate_variables[i - 1]
        if i != len(layers) - 1:
            s = auxiliary_variables[i]
            a = decision_variables[i]
            y = intermediate_variables[i]
        else:
            y = output_variables

        for j in range(A.shape[0]): # para cada neuronio da camada
            if i != len(layers) - 1:  # se não for a última camada(camada de saída)
                m_less, m_more = bounds[j]
                if m_more <= 0:
                    mdl.add_constraint(y[j] == 0)
                    continue

                if m_less >= 0:
                    mdl.add_constraint(A[j, :] @ x + b[j] == y[j])
                    continue

                if m_less < 0 and m_more > 0:
                    mdl.add_constraint(
                        A[j, :] @ x + b[j] == y[j] - s[j], ctname=f"c_{i}_{j}"
                    )
                    mdl.add_constraint(y[j] <= m_more * (1 - a[j]))
                    mdl.add_constraint(s[j] <= -m_less * a[j])
                    continue

            else:
                lb, ub = output_bounds_binary_variables[j]
                output_bounds.append([lb, ub])

    return mdl, output_bounds


def codify_network_tjeng(
    mdl,
    layers,
    input_variables,
    intermediate_variables,
    decision_variables,
    output_variables,
):
    output_bounds = []

    for i in range(len(layers)):
        A = layers[i].get_weights()[0].T
        b = layers[i].bias.numpy()
        x = input_variables if i == 0 else intermediate_variables[i - 1]
        if i != len(layers) - 1:
            a = decision_variables[i]
            y = intermediate_variables[i]
        else:
            y = output_variables

        for j in range(A.shape[0]):
            mdl.maximize(A[j, :] @ x + b[j])
            mdl.solve()
            ub = mdl.solution.get_objective_value()
            mdl.remove_objective()

            if ub <= 0 and i != len(layers) - 1:
                print("ENTROU, o ub é negativo, logo y = 0")
                mdl.add_constraint(y[j] == 0, ctname=f"c_{i}_{j}")
                continue

            mdl.minimize(A[j, :] @ x + b[j])
            mdl.solve()
            lb = mdl.solution.get_objective_value()
            mdl.remove_objective()

            if lb >= 0 and i != len(layers) - 1:
                print("ENTROU, o lb >= 0, logo y = Wx + b")
                mdl.add_constraint(A[j, :] @ x + b[j] == y[j], ctname=f"c_{i}_{j}")
                continue

            if i != len(layers) - 1:
                mdl.add_constraint(y[j] <= A[j, :] @ x + b[j] - lb * (1 - a[j]))
                mdl.add_constraint(y[j] >= A[j, :] @ x + b[j])
                mdl.add_constraint(y[j] <= ub * a[j])

                # mdl.maximize(y[j])
                # mdl.solve()
                # ub_y = mdl.solution.get_objective_value()
                # mdl.remove_objective()
                # y[j].set_ub(ub_y)

            else:
                mdl.add_constraint(A[j, :] @ x + b[j] == y[j])
                # y[j].set_ub(ub)
                # y[j].set_lb(lb)
                output_bounds.append([lb, ub])

    return mdl, output_bounds


def codify_network(model, dataframe, method, relaxe_constraints):
    layers = model.layers
    num_features = layers[0].get_weights()[0].shape[0]
    mdl = mp.Model()

    domain_input, bounds_input = get_domain_and_bounds_inputs(dataframe)
    bounds_input = np.array(bounds_input)

    if relaxe_constraints:
        input_variables = mdl.continuous_var_list(
            num_features, lb=bounds_input[:, 0], ub=bounds_input[:, 1], name="x"
        )
    else:
        input_variables = []
        for i in range(len(domain_input)):
            lb, ub = bounds_input[i]
            if domain_input[i] == "C":
                input_variables.append(mdl.continuous_var(lb=lb, ub=ub, name=f"x_{i}"))
            elif domain_input[i] == "I":
                input_variables.append(mdl.integer_var(lb=lb, ub=ub, name=f"x_{i}"))
            elif domain_input[i] == "B":
                input_variables.append(mdl.binary_var(name=f"x_{i}"))

    intermediate_variables = []
    auxiliary_variables = []
    decision_variables = []

    for i in range(len(layers) - 1):
        weights = layers[i].get_weights()[0]
        intermediate_variables.append(
            mdl.continuous_var_list(
                weights.shape[1], lb=0, name="y", key_format=f"_{i}_%s"
            )
        )

        if method == "fischetti":
            auxiliary_variables.append(
                mdl.continuous_var_list(
                    weights.shape[1], lb=0, name="s", key_format=f"_{i}_%s"
                )
            )

        if relaxe_constraints and method == "tjeng":
            decision_variables.append(
                mdl.continuous_var_list(
                    weights.shape[1], name="a", lb=0, ub=1, key_format=f"_{i}_%s"
                )
            )
        else:
            decision_variables.append(
                mdl.binary_var_list(
                    weights.shape[1], name="a", lb=0, ub=1, key_format=f"_{i}_%s"
                )
            )

    output_variables = mdl.continuous_var_list(
        layers[-1].get_weights()[0].shape[1], lb=-infinity, name="o"
    )

    if method == "tjeng":
        mdl, output_bounds = codify_network_tjeng(
            mdl,
            layers,
            input_variables,
            intermediate_variables,
            decision_variables,
            output_variables,
        )
    else:
        mdl, output_bounds, bounds = codify_network_fischetti(
            mdl,
            layers,
            input_variables,
            auxiliary_variables,
            intermediate_variables,
            decision_variables,
            output_variables,
        )

    if relaxe_constraints:
        # Tighten domain of variables 'a'
        for i in decision_variables:
            for a in i:
                a.set_vartype("Integer")

        # Tighten domain of input variables
        for i, x in enumerate(input_variables):
            if domain_input[i] == "I":
                x.set_vartype("Integer")
            elif domain_input[i] == "B":
                x.set_vartype("Binary")
            elif domain_input[i] == "C":
                x.set_vartype("Continuous")

    return mdl, output_bounds, bounds


def get_domain_and_bounds_inputs(dataframe):
    domain = []
    bounds = []
    for column in dataframe.columns[:-1]:
        if len(dataframe[column].unique()) == 2:
            domain.append("B")
            bound_inf = dataframe[column].min()
            bound_sup = dataframe[column].max()
            bounds.append([bound_inf, bound_sup])
        elif np.any(
            dataframe[column].unique().astype(np.int64)
            != dataframe[column].unique().astype(np.float64)
        ):
            domain.append("C")
            bound_inf = dataframe[column].min()
            bound_sup = dataframe[column].max()
            bounds.append([bound_inf, bound_sup])
        else:
            domain.append("I")
            bound_inf = dataframe[column].min()
            bound_sup = dataframe[column].max()
            bounds.append([bound_inf, bound_sup])

    return domain, bounds


def codify_network_relaxed(
    model, dataframe, method, relaxe_constraints, output_bounds_binary_variables, bounds
):
    layers = model.layers
    num_features = layers[0].get_weights()[0].shape[0]
    mdl = mp.Model()

    domain_input, bounds_input = get_domain_and_bounds_inputs(dataframe)
    bounds_input = np.array(bounds_input)

    if relaxe_constraints:
        input_variables = mdl.continuous_var_list(
            num_features, lb=bounds_input[:, 0], ub=bounds_input[:, 1], name="x"
        )
    else:
        input_variables = []
        for i in range(len(domain_input)):
            lb, ub = bounds_input[i]
            if domain_input[i] == "C":
                input_variables.append(mdl.continuous_var(lb=lb, ub=ub, name=f"x_{i}"))
            elif domain_input[i] == "I":
                input_variables.append(mdl.integer_var(lb=lb, ub=ub, name=f"x_{i}"))
            elif domain_input[i] == "B":
                input_variables.append(mdl.binary_var(name=f"x_{i}"))

    intermediate_variables = []
    auxiliary_variables = []
    decision_variables = []

    for i in range(len(layers) - 1):
        weights = layers[i].get_weights()[0]
        intermediate_variables.append(
            mdl.continuous_var_list(
                weights.shape[1], lb=0, name="y", key_format=f"_{i}_%s"
            )
        )

        if method == "fischetti":
            auxiliary_variables.append(
                mdl.continuous_var_list(
                    weights.shape[1], lb=0, name="s", key_format=f"_{i}_%s"
                )
            )

        if relaxe_constraints and method == "tjeng":
            decision_variables.append(
                mdl.continuous_var_list(
                    weights.shape[1], name="a", lb=0, ub=1, key_format=f"_{i}_%s"
                )
            )
        else:
            # decision_variables.append(mdl.binary_var_list(weights.shape[1], name='a', lb=0, ub=1, key_format=f"_{i}_%s"))
            decision_variables.append(
                mdl.continuous_var_list(
                    weights.shape[1], name="a", lb=0, ub=1, key_format=f"_{i}_%s"
                )
            )

    output_variables = mdl.continuous_var_list(
        layers[-1].get_weights()[0].shape[1], lb=-infinity, name="o"
    )

    if method == "tjeng":
        # modificar depois para utilizar bounds precisos
        mdl, output_bounds = codify_network_tjeng(
            mdl,
            layers,
            input_variables,
            intermediate_variables,
            decision_variables,
            output_variables,
        )
    else:
        # mdl, output_bounds = codify_network_fischetti(mdl, layers, input_variables, auxiliary_variables, intermediate_variables, decision_variables, output_variables)
        mdl, output_bounds = codify_network_fischetti_relaxed(
            mdl,
            layers,
            input_variables,
            auxiliary_variables,
            intermediate_variables,
            decision_variables,
            output_variables,
            output_bounds_binary_variables,
            bounds = bounds
        )

    if relaxe_constraints:
        # Tighten domain of variables 'a'
        for i in decision_variables:
            for a in i:
                # a.set_vartype('Integer')
                a.set_vartype("Continuous")

        # Tighten domain of input variables
        for i, x in enumerate(input_variables):
            if domain_input[i] == "I":
                x.set_vartype("Integer")
            elif domain_input[i] == "B":
                x.set_vartype("Binary")
            elif domain_input[i] == "C":
                x.set_vartype("Continuous")

    return mdl, output_bounds


# if __name__ == '__main__':
#     path_dir = 'glass'
#     #model = tf.keras.models.load_model(f'datasets\\{path_dir}\\model_{path_dir}.h5')
#     model = tf.keras.models.load_model(f'datasets\\{path_dir}\\teste.h5')

#     data_test = pd.read_csv(f'datasets\\{path_dir}\\test.csv')
#     data_train = pd.read_csv(f'datasets\\{path_dir}\\train.csv')
#     data = data_train._append(data_test)
#     data = data[['RI', 'Na', 'target']]

#     mdl, bounds = codify_network(model, data, 'tjeng', False)
#     print(mdl.export_to_string())
#     # print(bounds)
#     print("Bounds:")
#     for bound in bounds:
#         print(bound)

# X ---- E
# x1 == 1 /\ x2 == 3 /\ F /\ ~E    INSATISFÁTIVEL
# x1 >= 0 /\ x1 <= 100 /\ x2 == 3 /\ F /\ ~E    INSATISFÁTIVEL -> x1 n é relevante,  SATISFÁTIVEL -> x1 é relevante
"""
print("\n\nSolving model....\n")

msol = mdl.solve(log_output=True)
print(mdl.get_solve_status())
"""


'\nprint("\n\nSolving model....\n")\n\nmsol = mdl.solve(log_output=True)\nprint(mdl.get_solve_status())\n'

# Teste

In [28]:
from typing import List
import numpy as np
import tensorflow as tf
from milp import codify_network
from time import time
from statistics import mean, stdev
import pandas as pd
from docplex.mp.constr import LinearConstraint

# todo: ver se faz uma chamada para cada classe não predita
def insert_output_constraints_fischetti(
    mdl, output_variables, network_output, binary_variables
):
    variable_output = output_variables[network_output]
    aux_var = 0

    for i, output in enumerate(output_variables):
        if i != network_output:
            p = binary_variables[aux_var]
            aux_var += 1
            mdl.add_indicator(p, variable_output <= output, 1)

    return mdl


def insert_output_constraints_tjeng(
    mdl, output_variables, network_output, binary_variables, output_bounds
):
    variable_output = output_variables[network_output]
    upper_bounds_diffs = (
        output_bounds[network_output][1] - np.array(output_bounds)[:, 0]
    )  # Output i: oi - oj <= u1 = ui - lj
    aux_var = 0

    for i, output in enumerate(output_variables):
        if i != network_output:
            ub = upper_bounds_diffs[i]
            z = binary_variables[aux_var]
            mdl.add_constraint(variable_output - output - ub * (1 - z) <= 0)
            aux_var += 1

    return mdl


def get_minimal_explanation(
    mdl,
    network_input,
    network_output,
    n_classes,
    method,
    output_bounds=None,
    initial_explanation=None,
) -> List[LinearConstraint]:
    assert not (
        method == "tjeng" and output_bounds == None
    ), "If the method tjeng is chosen, output_bounds must be passed."

    output_variables = [mdl.get_var_by_name(f"o_{i}") for i in range(n_classes)]

    if initial_explanation is None:
        input_constraints = mdl.add_constraints(
            [
                mdl.get_var_by_name(f"x_{i}") == feature.numpy()
                for i, feature in enumerate(network_input[0])
            ],
            names="input",
        )
    else:
        input_constraints = mdl.add_constraints(
            [
                mdl.get_var_by_name(f"x_{i}") == network_input[0][i].numpy()
                for i in initial_explanation
            ],
            names="input",
        )

    binary_variables = mdl.binary_var_list(n_classes - 1, name="b") # todo: como isso é utilizado dentro do insert_output_constraints_fischetti?
    mdl.add_constraint(mdl.sum(binary_variables) >= 1)

    if method == "tjeng":
        mdl = insert_output_constraints_tjeng(
            mdl, output_variables, network_output, binary_variables, output_bounds
        )
    else:
        mdl = insert_output_constraints_fischetti(
            mdl, output_variables, network_output, binary_variables
        )

    for constraint in input_constraints:
        mdl.remove_constraint(constraint)

        mdl.solve(log_output=False)
        if mdl.solution is not None: 
            mdl.add_constraint(constraint)

    return mdl.find_matching_linear_constraints("input")


def get_explanation_relaxed(
    mdl,
    network_input,
    network_output,
    n_classes,
    method,
    output_bounds=None,
    initial_explanation=None,
    delta=0.1,
) -> List[LinearConstraint]:
    # todo: output_bounds só é relevante se o metodo for tjeng
    assert not (
        method == "tjeng" and output_bounds == None
    ), "If the method tjeng is chosen, output_bounds must be passed."

    output_variables = [mdl.get_var_by_name(f"o_{i}") for i in range(n_classes)]

    if initial_explanation is None:
        input_constraints = mdl.add_constraints(
            [
                mdl.get_var_by_name(f"x_{i}") == feature.numpy()
                for i, feature in enumerate(network_input[0])
            ],
            names="input",
        )
    else:
        input_constraints = mdl.add_constraints(
            [
                mdl.get_var_by_name(f"x_{i}") == network_input[0][i].numpy()
                for i in initial_explanation
            ],
            names="input",
        )

    binary_variables = mdl.binary_var_list(n_classes - 1, name="b")
    mdl.add_constraint(mdl.sum(binary_variables) >= 1)

    if method == "tjeng":
        mdl = insert_output_constraints_tjeng(
            mdl, output_variables, network_output, binary_variables, output_bounds
        )

    # todo: !(o1>o2 and o1>o3)
    # todo: modificar para o1<=o2 or o1<=o3
    else:
        mdl = insert_output_constraints_fischetti(
            mdl, output_variables, network_output, binary_variables
        )

    for constraint in input_constraints:
        mdl.remove_constraint(constraint)

        x = constraint.get_left_expr()
        v = constraint.get_right_expr()

        constraint_left = mdl.add_constraint(v - delta <= x)
        constraint_right = mdl.add_constraint(x <= v + delta)

        mdl.solve(log_output=False)
        if mdl.solution is not None:
            mdl.add_constraint(constraint)
            mdl.remove_constraint(constraint_left)
            mdl.remove_constraint(constraint_right)

    return mdl.find_matching_linear_constraints("input")


def main():
    datasets = [  # {'dir_path': 'australian', 'n_classes': 2},
        # {'dir_path': 'auto', 'n_classes': 5},
        # {'dir_path': 'backache', 'n_classes': 2},
        # {'dir_path': 'breast-cancer', 'n_classes': 2},
        # {'dir_path': 'cleve', 'n_cla
        # sses': 2},
        # {'dir_path': 'cleveland', 'n_classes': 5},
        # {'dir_path': 'glass', 'n_classes': 5},
        {"dir_path": "glass2", "n_classes": 2},
        # {'dir_path': 'heart-statlog', 'n_classes': 2}, {'dir_path': 'hepatitis', 'n_classes': 2},
        # {'dir_path': 'spect', 'n_classes': 2},
        # {'dir_path': 'voting', 'n_classes': 2}
    ]

    configurations = [  # {'method': 'fischetti', 'relaxe_constraints': True},
        {"method": "fischetti", "relaxe_constraints": True},
        # {'method': 'tjeng', 'relaxe_constraints': True},
        {"method": "tjeng", "relaxe_constraints": False},
    ]

    df = {
        "fischetti": {
            True: {"size": [], "milp_time": [], "build_time": []},
            False: {"size": [], "milp_time": [], "build_time": []},
        },
        "tjeng": {
            True: {"size": [], "milp_time": [], "build_time": []},
            False: {"size": [], "milp_time": [], "build_time": []},
        },
    }

    for dataset in datasets:
        dir_path = dataset["dir_path"]
        n_classes = dataset["n_classes"]

        for config in configurations:
            print(dataset, config)

            method = config["method"]
            relaxe_constraints = config["relaxe_constraints"]

            data_test = pd.read_csv(f"datasets\\{dir_path}\\test.csv")
            data_train = pd.read_csv(f"datasets\\{dir_path}\\train.csv")
            data = data_train._append(data_test)

            model_path = f"datasets\\{dir_path}\\model_4layers_{dir_path}.h5"
            model = tf.keras.models.load_model(model_path)

            codify_network_time = []
            for _ in range(10):
                start = time()
                mdl, output_bounds = codify_network(
                    model, data, method, relaxe_constraints
                )
                codify_network_time.append(time() - start)
                print(codify_network_time[-1])

            time_list = []
            len_list = []
            # data = data.to_numpy()
            data = data_test.to_numpy()
            for i in range(data.shape[0]):
                # if i % 50 == 0:
                print(i)
                network_input = data[i, :-1]

                network_input = tf.reshape(tf.constant(network_input), (1, -1))
                network_output = model.predict(tf.constant(network_input))[0]
                network_output = tf.argmax(network_output)

                mdl_aux = mdl.clone()
                start = time()

                explanation = get_minimal_explanation(
                    mdl_aux,
                    network_input,
                    network_output,
                    n_classes=n_classes,
                    method=method,
                    output_bounds=output_bounds,
                )

                time_list.append(time() - start)

                len_list.append(len(explanation))

            df[method][relaxe_constraints]["size"].extend(
                [min(len_list), f"{mean(len_list)} +- {stdev(len_list)}", max(len_list)]
            )
            df[method][relaxe_constraints]["milp_time"].extend(
                [
                    min(time_list),
                    f"{mean(time_list)} +- {stdev(time_list)}",
                    max(time_list),
                ]
            )
            df[method][relaxe_constraints]["build_time"].extend(
                [
                    min(codify_network_time),
                    f"{mean(codify_network_time)} +- {stdev(codify_network_time)}",
                    max(codify_network_time),
                ]
            )

            print(
                f"Explication sizes:\nm: {min(len_list)}\na: {mean(len_list)} +- {stdev(len_list)}\nM: {max(len_list)}"
            )
            print(
                f"Time:\nm: {min(time_list)}\na: {mean(time_list)} +- {stdev(time_list)}\nM: {max(time_list)}"
            )
            print(
                f"Build Time:\nm: {min(codify_network_time)}\na: {mean(codify_network_time)} +- {stdev(codify_network_time)}\nM: {max(codify_network_time)}"
            )
    "a" + 1
    df = {
        "fischetti_relaxe_size": df["fischetti"][True]["size"],
        "fischetti_relaxe_time": df["fischetti"][True]["milp_time"],
        "fischetti_relaxe_build_time": df["fischetti"][True]["build_time"],
        "fischetti_not_relaxe_size": df["fischetti"][False]["size"],
        "fischetti_not_relaxe_time": df["fischetti"][False]["milp_time"],
        "fischetti_not_relaxe_build_time": df["fischetti"][False]["build_time"],
        "tjeng_relaxe_size": df["tjeng"][True]["size"],
        "tjeng_relaxe_time": df["tjeng"][True]["milp_time"],
        "tjeng_relaxe_build_time": df["tjeng"][True]["build_time"],
        "tjeng_not_relaxe_size": df["tjeng"][False]["size"],
        "tjeng_not_relaxe_time": df["tjeng"][False]["milp_time"],
        "tjeng_not_relaxe_build_time": df["tjeng"][False]["build_time"],
    }

    index_label = []
    for dataset in datasets:
        index_label.extend(
            [
                f"{dataset['dir_path']}_m",
                f"{dataset['dir_path']}_a",
                f"{dataset['dir_path']}_M",
            ]
        )
    df = pd.DataFrame(data=df, index=index_label)
    df.to_csv("results.csv")


# Main


In [29]:
import os
from time import time
import pandas as pd
import tensorflow as tf
from milp import codify_network, codify_network_relaxed
from teste import get_explanation_relaxed, get_minimal_explanation
from typing import List
from docplex.mp.constr import LinearConstraint


In [30]:
def gerar_rede(dir_path: str, num_classes: int, n_neurons: int, n_hidden_layers: int):
    data_train = pd.read_csv(dir_path + "\\" + "train.csv").to_numpy()
    data_test = pd.read_csv(dir_path + "\\" + "test.csv").to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons.h5"
    )

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    ck = tf.keras.callbacks.ModelCheckpoint(
        model_path, monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    print(f"Tempo de Treinamento: {time()-start}")

    # salvar modelo
    model = tf.keras.models.load_model(model_path)

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    model.evaluate(x_test, y_test_ohe, verbose=2)

In [31]:
def gerar_rede_com_dataset_iris(n_neurons=20, n_hidden_layers=1):
    dir_path = "datasets\\iris"
    num_classes = 3
    gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)


def gerar_rede_com_dataset_digits(n_neurons=20, n_hidden_layers=1):
    dir_path = "datasets\\digits"
    num_classes = 10
    gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)


def gerar_rede_com_dataset_wine(n_neurons=20, n_hidden_layers=1):
    dir_path = "datasets\\wine"
    num_classes = 10
    gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)

In [32]:
def explain_instance(initial_network,
    dataset: {}, configuration: {}, instance_index: int
) -> List[LinearConstraint]:
    dir_path, n_classes, model = (
        dataset["dir_path"],
        dataset["n_classes"],
        dataset["model"],
    )

    method = configuration["method"]
    relaxe_constraints = configuration["relaxe_constraints"]

    data_test = pd.read_csv(f"{dir_path}/test.csv")
    data_train = pd.read_csv(f"{dir_path}/train.csv")

    data = data_train._append(data_test)
    model = tf.keras.models.load_model(f"{dir_path}/{model}")
    
    (
        mdl_milp_with_binary_variable,
        output_bounds_binary_variables,
        bounds,
    ) = initial_network
        
    
    # todo: ao inves de receber um, receber varios a serem explicados sem ter que codificar uma nova rede
    # todo: salvar a rede de alguma forma para reutilizar
    network_input = data.iloc[instance_index, :-1]
    # print(network_input)  # network_input = instance

    network_input = tf.reshape(tf.constant(network_input), (1, -1))

    network_output = model.predict(tf.constant(network_input))[0]

    network_output = tf.argmax(network_output)
    mdl_aux = mdl_milp_with_binary_variable.clone()

    explanation = get_minimal_explanation(
        mdl_aux,
        network_input,
        network_output,
        n_classes=n_classes,
        method=method,
        output_bounds=output_bounds_binary_variables,
    )
    return explanation


In [33]:
def explain_instance_relaxed(
    initial_network,
    initial_network_relaxed,
    dataset: {},
    configuration: {},
    instance_index: int,
    delta=1,
) -> List[LinearConstraint]:
    dir_path, n_classes, model = (
        dataset["dir_path"],
        dataset["n_classes"],
        dataset["model"],
    )

    method = configuration["method"]
    relaxe_constraints = configuration["relaxe_constraints"]

    data_test = pd.read_csv(f"{dir_path}/test.csv")
    data_train = pd.read_csv(f"{dir_path}/train.csv")

    data = data_train._append(data_test)

    model = tf.keras.models.load_model(f"{dir_path}/{model}")
    
    (
        mdl_milp_with_binary_variable,
        output_bounds_binary_variables,
        bounds,
    ) = initial_network
    
    model_milp_relaxed, output_bounds_relaxed = initial_network_relaxed


    network_input = data.iloc[instance_index, :-1]
    # print(network_input)  # network_input = instance

    network_input = tf.reshape(tf.constant(network_input), (1, -1))

    network_output = model.predict(tf.constant(network_input))[0]

    network_output = tf.argmax(network_output)

    mdl_aux = model_milp_relaxed.clone()

    explanation = get_explanation_relaxed(
        mdl_aux,
        network_input,
        network_output,
        n_classes=n_classes,
        method=method,
        output_bounds=output_bounds_binary_variables,  # output_bounds_binary_variables == output_bounds_relaxed
        delta=delta,
    )

    return explanation

In [34]:
# def explicar_rede():
#     datasets = [
#         {
#             "dir_path": "datasets/digits",
#             "model": "models/model_1layers_20neurons.h5",
#             "n_classes": 10,
#         },
#         {
#             "dir_path": "datasets/iris",
#             "model": "models/model_1layers_20neurons.h5",
#             "n_classes": 3,
#         },
#         {
#             "dir_path": "datasets/iris",
#             "model": "models/model_6layers_20neurons.h5",
#             "n_classes": 3,
#         },
#     ]
#     configurations = [{"method": "fischetti", "relaxe_constraints": True}]
    
#     dataset_index = 0

#     for i in range(0, 1):
#         print("binaria", end = '')
#         explanation = explain_instance(
#             dataset=datasets[dataset_index], configuration=configurations[0], instance_index=i
#         )

#         # for x in explanation:
#         #     print(x)
#         print("len: ", len(explanation), "\n")

        
#         print("relaxada", end = '')
#         explanation = explain_instance_relaxed(
#             dataset=datasets[dataset_index], configuration=configurations[0], instance_index=i, delta = 0.5
#         )

#         # for x in explanation:
#         #     print(x)
#         print("len: ", len(explanation), "\n")
        

In [35]:
# explicar_rede()

In [36]:
datasets = [
        {
            "dir_path": "datasets/digits",
            "model": "models/model_1layers_20neurons.h5",
            "n_classes": 10,
        },
        {
            "dir_path": "datasets/digits",
            "model": "models/model_2layers_20neurons.h5",
            "n_classes": 10,
        },{
            "dir_path": "datasets/digits",
            "model": "models/model_3layers_20neurons.h5",
            "n_classes": 10,
        },{
            "dir_path": "datasets/digits",
            "model": "models/model_4layers_20neurons.h5",
            "n_classes": 10,
        },{
            "dir_path": "datasets/digits",
            "model": "models/model_5layers_20neurons.h5",
            "n_classes": 10,
        },
        {
            "dir_path": "datasets/iris",
            "model": "models/model_1layers_20neurons.h5",
            "n_classes": 3,
        },
        {
            "dir_path": "datasets/iris",
            "model": "models/model_6layers_20neurons.h5",
            "n_classes": 3,
        },
    ]
configurations = [{"method": "fischetti", "relaxe_constraints": True}]


## Benchmark

Codificar Redes

In [37]:
import time
dataset_index = 3

dir_path, n_classes, model = (
    datasets[dataset_index]["dir_path"],
    datasets[dataset_index]["n_classes"],
    datasets[dataset_index]["model"],
)

method = configurations[0]["method"]
relaxe_constraints = configurations[0]["relaxe_constraints"]
data_test = pd.read_csv(f"{dir_path}/test.csv")
data_train = pd.read_csv(f"{dir_path}/train.csv")
data = data_train._append(data_test)
model = tf.keras.models.load_model(f"{dir_path}/{model}")

In [38]:
initial_network = codify_network(model, data, method, relaxe_constraints)
(
    mdl_milp_with_binary_variable,
    output_bounds_binary_variables,
    bounds,
) = initial_network

In [39]:
initial_network_relaxed = codify_network_relaxed(
    model,
    data,
    method,
    relaxe_constraints,
    output_bounds_binary_variables,
    bounds=bounds,
)

In [40]:
import pandas as pd
caminho_do_diretorio = "C:\\Users\\mylle\\OneDrive\\Documentos\\GitHub\\TestesTypescript\\Explications-ANNs"
nome_do_arquivo = f"{datasets[dataset_index]['dir_path']}/{datasets[dataset_index]['model']}_resultados.csv"
caminho_completo = f"{caminho_do_diretorio}/{nome_do_arquivo}"

if os.path.isfile(caminho_completo):
    # Se existe, leia o CSV
    resultados = pd.read_csv(caminho_completo)
else:
    # Se não existe, crie um DataFrame vazio
    resultados = pd.DataFrame(columns=["instance_index", "tempo_original", "tempo_relaxado", "len_original", "len_relaxado", "delta"])

In [46]:
len_resultados = len(resultados)
qtd = 10
delta = 0.5

def limpar_console():
    os.system('clear')

for instance_index in range(len_resultados, len_resultados+qtd):
    print(instance_index)
    # explain_instance
    start_time = time.time()
    explanation = explain_instance(
        initial_network=initial_network,
        dataset=datasets[dataset_index],
        configuration=configurations[0],
        instance_index=instance_index,
    )
    end_time = time.time()
    tempo_original = end_time - start_time
    len_original = len(explanation)

    # explain_instance_relaxed
    start_time = time.time()
    explanation_relaxed = explain_instance_relaxed(
        initial_network=initial_network,
        initial_network_relaxed=initial_network_relaxed,
        dataset=datasets[dataset_index],
        configuration=configurations[0],
        instance_index=instance_index,
        delta=delta,
    )
    end_time = time.time()
    tempo_relaxado = end_time - start_time
    len_relaxado = len(explanation_relaxed)
    resultados.loc[len(resultados)] = [
        instance_index,
        tempo_original,
        tempo_relaxado,
        len_original,
        len_relaxado,
        delta
    ]
    # salvar
    resultados.to_csv(caminho_completo, index=False)

9
1/1 [==============================] - 0s 79ms/step
10
1/1 [==============================] - 0s 215ms/step
11
1/1 [==============================] - 0s 97ms/step
12
1/1 [==============================] - 0s 115ms/step
13
1/1 [==============================] - 0s 229ms/step
14
1/1 [==============================] - 0s 92ms/step
15
1/1 [==============================] - 0s 79ms/step
16
1/1 [==============================] - 0s 79ms/step
17
1/1 [==============================] - 0s 367ms/step
18
1/1 [==============================] - 0s 228ms/step


Salvar o DataFrame como CSV

In [ ]:
# resultados.to_csv(caminho_completo, index=False)

Filtrar linhas

In [ ]:
filtrado = resultados[resultados['len_relaxado'] < 64]

In [ ]:
print(len(resultados))
print(len(filtrado))
len_r = len(resultados)
len_f = len(filtrado)

9
9


In [ ]:
len_f*100/len_r

100.0